In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tqdm.keras import TqdmCallback
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, BatchNormalization
from sklearn.model_selection import train_test_split

In [ ]:
base_path = '../input/face-images-with-marked-landmark-points/'

In [ ]:
df = pd.read_csv(base_path + 'facial_keypoints.csv')
df.head()

In [ ]:
df = df.fillna(0)

In [ ]:
c = 0
columns = {}
temp = []
for i, e in enumerate(list(df.columns)):
    temp.append(e)
    c += 1
    if c == 2:
        columns[e.split('_y')[0]] = temp
        temp = []
        c = 0

In [ ]:
keypoints_dict = {}
for k in columns.keys():
    keypoints_dict[k] = df[columns[k]].values

keypoints = np.array(list(keypoints_dict.values()))
keypoints = np.swapaxes(keypoints, 0, 1)
features = np.load(base_path + 'face_images.npz')['face_images']
features = np.swapaxes(np.swapaxes(features, 1, 2), 0, 1)    

In [ ]:
keypoints = keypoints.reshape(-1, 30)
features = features.reshape(7049, 96, 96, 1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, keypoints, test_size=0.1)
img_size = 96

In [ ]:
model = Sequential()
model.add(Input(shape=(img_size, img_size, 1)))
model.add(BatchNormalization())
model.add(Conv2D(16, (3,3), padding="same",kernel_initializer=glorot_uniform()))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding="same",kernel_initializer=glorot_uniform()))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding="same",kernel_initializer=glorot_uniform()))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))  
model.add(Flatten())
model.add(Dense(128,kernel_initializer=glorot_uniform()))
model.add(LeakyReLU(alpha=0.1))
model.add(Dropout(0.5)) 
model.add(Dense(64,kernel_initializer=glorot_uniform()))
model.add(LeakyReLU(alpha=0))
model.add(Dense(30,kernel_initializer=glorot_uniform()))
model.compile(loss='mean_squared_error', optimizer=Adam(), metrics=['mean_squared_error'])

In [ ]:
BATCH_SIZE = 32
EPOCHS = 1000

history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(X_test, y_test),
    shuffle=True,
    verbose=0,
    callbacks=[TqdmCallback(verbose=0)]
)